In [1]:
!pip install -U bitsandbytes

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.0/67.0 MB 9.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 43.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 35.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 29.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 11.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 5.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 7.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 35.9 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstalling

In [2]:
# ✅ Step 1: Install dependencies
!pip install -q transformers accelerate torch torchvision sentencepiece bitsandbytes

# ✅ Step 2: Import modules
from transformers import AutoProcessor, AutoModelForVision2Seq
from PIL import Image
import torch
import re
from google.colab import files
from io import BytesIO
from transformers import BitsAndBytesConfig

# ✅ Step 3: Upload image
uploaded = files.upload()
image_path = list(uploaded.keys())[0]
image = Image.open(image_path).convert("RGB")

# ✅ Step 4: Load LLaVA model
# Using a smaller model and quantization to avoid OutOfMemoryError
model_id = "llava-hf/llava-1.5-7b-hf"

# Configure quantization
quantization_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.float16,
    bnb_4bit_use_double_quant=True,
)

processor = AutoProcessor.from_pretrained(model_id)
model = AutoModelForVision2Seq.from_pretrained(
    model_id,
    device_map="auto",
    torch_dtype=torch.float16,
    quantization_config=quantization_config,
)

# ✅ Step 5: Define prompt and inference
prompt = "How many pests are visible in this image? Please provide the number as a single digit or number."

# Format the prompt with the image token
chat_template = "{% for message in messages %}{% if message['role'] == 'user' %}{{ message['content'] | join }}{% elif message['role'] == 'assistant' %}{{ message['content'] }}{% endif %}{% if loop.last and add_generation_prompt %}{{ '<|endoftext|>' }}{% endif %}{% endfor %}"

messages = [
    {"role": "user", "content": [{"type": "image"}, {"type": "text", "text": prompt}]}
]

text_input = processor.apply_chat_template(
    messages,
    tokenize=False,
    add_generation_prompt=True,
)

inputs = processor(text=text_input, images=image, return_tensors="pt").to(model.device)
generate_ids = model.generate(**inputs, max_new_tokens=256)
output_text = processor.batch_decode(generate_ids, skip_special_tokens=True)[0]

# ✅ Step 6: Extract count
match = re.search(r'\b(\d{1,3})\b', output_text)
count = int(match.group(1)) if match else None
leaf_area_cm2 = 20
density = count / leaf_area_cm2 if count else "N/A"

# ✅ Step 7: Final Output
print("\n📷 Pest Image Analysis (LLaVA 1.5)")
print(f"🧾 Model Output: {output_text}")
if count:
    print(f"✅ Pests Detected: Yes ({count})")
    print(f"🌿 Estimated Density: {density:.2f} pests/cm²")
else:
    print("⚠️ Pest count not clearly extracted.")

Saving P (1).jpg to P (1).jpg


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


processor_config.json:   0%|          | 0.00/173 [00:00<?, ?B/s]

chat_template.json:   0%|          | 0.00/701 [00:00<?, ?B/s]

chat_template.jinja:   0%|          | 0.00/674 [00:00<?, ?B/s]

...

model-00002-of-00003.safetensors:   0%|          | 0.00/4.96G [00:00<?, ?B/s]

model-00003-of-00003.safetensors:   0%|          | 0.00/4.18G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/141 [00:00<?, ?B/s]


📷 Pest Image Analysis (LLaVA 1.5)
🧾 Model Output: USER:  
How many pests are visible in this image? Please provide the number as a single digit or number. ASSISTANT: 10
✅ Pests Detected: Yes (10)
🌿 Estimated Density: 0.50 pests/cm²


In [1]:
# ✅ Step 1: Install dependencies
!pip install -q transformers accelerate torch torchvision sentencepiece bitsandbytes

# ✅ Step 2: Import modules
from transformers import AutoProcessor, AutoModelForVision2Seq, BitsAndBytesConfig
from PIL import Image, ImageDraw, ImageFont
import torch
import re
from google.colab import files
import pandas as pd
import os

# ✅ Step 3: Upload multiple images
uploaded = files.upload()
image_paths = list(uploaded.keys())

# ✅ Step 4: Load quantized LLaVA model
model_id = "llava-hf/llava-1.5-7b-hf"
quantization_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.float16,
    bnb_4bit_use_double_quant=True,
)
processor = AutoProcessor.from_pretrained(model_id)
model = AutoModelForVision2Seq.from_pretrained(
    model_id,
    device_map="auto",
    torch_dtype=torch.float16,
    quantization_config=quantization_config,
)

# ✅ Step 5: Prompt Template
prompt = (
    "Please examine the image and provide pest details in the format:\n"
    "Aphids: [number], Thrips: [number], Spider mites: [number], Others: [number].\n"
    "Only give numeric answers."
)

# ✅ Step 6: Analyze images
results = []
for image_path in image_paths:
    image = Image.open(image_path).convert("RGB")
    width, height = image.size
    dpi = 300
    pixels_per_cm = dpi / 2.54
    area_cm2 = (width / pixels_per_cm) * (height / pixels_per_cm)

    messages = [{"role": "user", "content": [{"type": "image"}, {"type": "text", "text": prompt}]}]
    text_input = processor.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)
    inputs = processor(text=text_input, images=image, return_tensors="pt").to(model.device)
    generate_ids = model.generate(**inputs, max_new_tokens=256)
    output_text = processor.batch_decode(generate_ids, skip_special_tokens=True)[0]

    # ✅ Extract counts for each pest type
    def extract_count(keyword):
        match = re.search(rf"{keyword}:\s*(\d+)", output_text, re.IGNORECASE)
        return int(match.group(1)) if match else 0

    aphids = extract_count("Aphids")
    thrips = extract_count("Thrips")
    mites = extract_count("Spider mites")
    others = extract_count("Others")
    total = aphids + thrips + mites + others
    density = total / area_cm2 if total else None

    # ✅ Annotate image
    annotated = image.copy()
    draw = ImageDraw.Draw(annotated)
    font = ImageFont.load_default()
    annotation_msg = f"Aphids: {aphids}, Thrips: {thrips}, Mites: {mites}, Others: {others}"
    draw.text((10, 10), annotation_msg, fill="red", font=font)
    annotated.save(f"annotated_{os.path.basename(image_path)}")

    results.append({
        "Image": image_path,
        "Leaf Area (cm²)": round(area_cm2, 2),
        "Aphids": aphids,
        "Thrips": thrips,
        "Spider Mites": mites,
        "Others": others,
        "Total Pests": total,
        "Density (pests/cm²)": f"{density:.2f}" if density else "N/A",
        "Model Output": output_text
    })

    print(f"\n📷 Analyzed: {image_path}")
    print(f"🧾 Model Output:\n{output_text}")
    print(f"✅ Total: {total}, Density: {f'{density:.2f}' if density else 'N/A'}")

# ✅ Step 7: Save results to CSV
df = pd.DataFrame(results)
df.to_csv("structured_pest_results.csv", index=False)
print("\n📊 Structured results saved to structured_pest_results.csv")


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 33.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 28.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 46.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 3.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 12.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 9.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 31.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.0/67.0 MB 10.4 MB/s eta 0:00:00


Saving 1 Oak Aphid Myzocalliss - Burr Oak  2023 1B.jpeg to 1 Oak Aphid Myzocalliss - Burr Oak  2023 1B.jpeg


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


processor_config.json:   0%|          | 0.00/173 [00:00<?, ?B/s]

chat_template.json:   0%|          | 0.00/701 [00:00<?, ?B/s]

...

model-00003-of-00003.safetensors:   0%|          | 0.00/4.18G [00:00<?, ?B/s]

model-00002-of-00003.safetensors:   0%|          | 0.00/4.96G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/141 [00:00<?, ?B/s]


📷 Analyzed: 1 Oak Aphid Myzocalliss - Burr Oak  2023 1B.jpeg
🧾 Model Output:
USER:  
Please examine the image and provide pest details in the format:
Aphids: [number], Thrips: [number], Spider mites: [number], Others: [number].
Only give numeric answers. ASSISTANT: Aphids: 100, Thrips: 0, Spider mites: 0, Others: 0.
✅ Total: 100, Density: 4.04

📊 Structured results saved to structured_pest_results.csv
